# Basic concepts IFP processing 

This notebook explains how to use Prolif to generate IFPs from Molecular Dynamics Simulation and how to restructure dataframes for further processing with IFPAggVis.
The notebook is based on an example data set provided in the publication, as well as open-source in Zenodo.

In [1]:
import MDAnalysis as mda
import prolif as plf
import pandas as pd


#### Assign vdW Radius
First, we have to assign a van der Waals Radius to manganese, as this is natively unknown in MDAnalysis.

In [2]:
vdw = mda.topology.tables.vdwradii.copy()
vdw["Mn"]=vdw["MG"]


Assign ligand name to variable and provide in/outpath

In [3]:
ligand = 1
file_path = "../data/md_data/"
outpath = "../data/csv_files/"

Load MD simulation files in MDAnalysis universe

In [4]:
file_rep1 = file_path + "replicate1/ligand" + str(ligand) + "/complex_cap2/analysis/wholetraj_nopbc_lig" + str(ligand) + "_1.xtc"
file_rep2 = file_path + "replicate2/ligand" + str(ligand) + "/complex_cap2/analysis/wholetraj_nopbc_lig" + str(ligand) + "_2.xtc"
file_rep3 = file_path + "replicate3/ligand" + str(ligand) + "/complex_cap2/analysis/wholetraj_nopbc_lig" + str(ligand) + "_3.xtc"

u = mda.Universe(file_path + "replicate1/ligand" + str(ligand) + '/complex_cap2/complex.gro',
                 file_rep1, file_rep2, file_rep3)


Assign bonds and elements to structure files and add to universe topology

In [5]:
guessed_bonds = mda.topology.guessers.guess_bonds(u.atoms.atoms, 
                                                  u.atoms.positions)
guessed_elements = mda.topology.guessers.guess_types(u.atoms.names)
u.add_TopologyAttr('elements', guessed_elements)
u.add_TopologyAttr('bonds', guessed_bonds)


### Process residues without vdW...

Assign name of ligand and atoms of protein to variable

In [6]:
lig = u.atoms.select_atoms("resname LG" + str(ligand))

prot = u.atoms.select_atoms("not resname LG" + str(ligand))


In [7]:
print(prot.residues)

<ResidueGroup [<Residue ACE, 6>, <Residue LEU, 7>, <Residue ASN, 8>, ..., <Residue MN, 401>, <Residue HOH, 402>, <Residue HOH, 8>]>


Use default interactions, remove VdWContact, since it is the only interaction detected if kept

In [8]:
interactions_all = plf.Fingerprint.list_available(show_hidden=True)
interactions_all.remove("VdWContact")
print(interactions_all)


['Anionic', 'CationPi', 'Cationic', 'EdgeToFace', 'FaceToFace', 'HBAcceptor', 'HBDonor', 'Hydrophobic', 'Interaction', 'MetalAcceptor', 'MetalDonor', 'PiCation', 'PiStacking', 'XBAcceptor', 'XBDonor', '_BaseCationPi', '_BaseHBond', '_BaseIonic', '_BaseMetallic', '_BaseXBond', '_Distance']


Process interaction fingerprint

In [9]:
fp = plf.Fingerprint(interactions=interactions_all)
# Run fingerprint on trajectory
fp.run(u.trajectory, lig, prot)



  0%|          | 0/841 [00:00<?, ?it/s]

<prolif.fingerprint.Fingerprint: 14 interactions: ['Hydrophobic', 'HBDonor', 'HBAcceptor', 'XBAcceptor', 'XBDonor', 'Cationic', 'Anionic', 'PiCation', 'CationPi', 'PiStacking', 'FaceToFace', 'EdgeToFace', 'MetalDonor', 'MetalAcceptor'] at 0x1da505d8820>

save to file

In [10]:
df = pd.DataFrame(fp.to_dataframe())


In [11]:
df.head(3)

ligand            LG11                                                         \
protein           HOH8       HIS66       ASP95       ARG96                      
interaction HBAcceptor Hydrophobic Hydrophobic Hydrophobic HBDonor HBAcceptor   
Frame                                                                           
0                 True       False       False        True   False       True   
100               True       False       False        True   False       True   
200               True       False       False        True   False       True   

ligand                                                   ...              \
protein                  GLY123      ASN124      HIS125  ...      CYS273   
interaction Anionic Hydrophobic Hydrophobic Hydrophobic  ... Hydrophobic   
Frame                                                    ...               
0              True       False       False        True  ...        True   
100            True       False       False        True  ...        True   
200            True       False       False        True  ...        True   

ligand                                                                        \
protein                     GLY274                 GLU275                      
interaction HBAcceptor Hydrophobic HBAcceptor Hydrophobic HBDonor HBAcceptor   
Frame                                                                          
0                False        True       True        True   False       True   
100               True       False      False        True   False      False   
200               True       False      False        True   False      False   

ligand                                       
protein                   PHE276     HOH402  
interaction Cationic Hydrophobic HBAcceptor  
Frame                                        
0              False       False      False  
100            False        True       True  
200            False        True       True  

[3 rows x 79 columns]

In [12]:
len(df)

841

save IFP df to file

In [12]:
df.to_csv(outpath+'Interactions_lig' + str(ligand) + '_allreps_wovdw.csv', sep=',')


### Process manganese ions with vdW...

since manganese ions might have a van der Waals interaction with MC-LR, we add the interaction seperately...
HOH added as atom, since RDKit requires hydrogen bonds which are not present with Mn, but will be removed later on.

In [13]:
mn = u.atoms.select_atoms("resname MN or resname HOH")
print(mn.residues)


<ResidueGroup [<Residue MN, 400>, <Residue MN, 401>, <Residue HOH, 402>, <Residue HOH, 8>]>


Define VdWContact interaction

In [14]:
interactions_all = ["VdWContact"]

Process interaction fingerprint for Mn ions and HOH

In [15]:
fp_mn = plf.Fingerprint(interactions=interactions_all)
fp_mn.run(u.trajectory, lig, mn)

  0%|          | 0/841 [00:00<?, ?it/s]

<prolif.fingerprint.Fingerprint: 1 interactions: ['VdWContact'] at 0x2268f569ca0>

In [16]:
df_mn = pd.DataFrame(fp_mn.to_dataframe())

In [17]:
df_mn.head(3)

ligand            LG11                                 
protein           HOH8      MN400      MN401     HOH402
interaction VdWContact VdWContact VdWContact VdWContact
Frame                                                  
0                 True      False      False       True
100               True      False       True       True
200               True      False      False       True

In [18]:
df_mn.to_csv(outpath + 'Interactions_lig' + str(ligand) + '_allreps_Mn.csv', sep=',')

### Process and Restructure IFP dataframes

In [21]:
from IFPAggVis.ifpaggvis import helpers

#### process and restructure interactions without VdWContact
remove multiindex and replace True/False with 0/1

In [22]:
df = df.droplevel("ligand", axis=1)
df_new = helpers.get_res_names_in_col_index(df)
df_new.replace({False: 0, True: 1}, inplace=True)    
df_new.to_csv(outpath + "ligand_" + str(ligand) + "_res_based_in_columns.csv")

#### process and restructure interactions with VdWContact
remove multiindex and replace True/False with 0/1

In [23]:
df_mn = df_mn.droplevel("ligand", axis=1)
df_new_mn = helpers.get_res_names_in_col_index(df_mn)
df_new_mn.replace({False: 0, True: 1}, inplace=True)    
df_new_mn.to_csv(outpath + "ligand_" + str(ligand) + "_res_based_in_columns.csv")

### Merge IFP dataframes

In [24]:
import re

columns of VdW Contact df to list to search them..

In [25]:
columns = df_new_mn.columns.values.tolist()

Select columns with manganese ions

In [26]:
selected_columns = [match for match in columns if "MN" in match] 

Merge IFP set of MN with initial IFP set of protein residues by adding vdWContact columns to major df

In [27]:
for col in selected_columns:
    if len(df_new) == len(df_new_mn):
        df_new[col] = df_new_mn[col].values
    else:
        print("Mismatch in number of rows! Double check your data.")


Sort columns according to residue number for further processing

In [28]:
sorted_cols = sorted(df_new.columns.values.tolist(), key=lambda s: int(re.search(r'\d+', s).group()))


In [29]:
sorted_cols

['HOH8_HBAcceptor',
 'HIS66_Hydrophobic',
 'ASP95_Hydrophobic',
 'ARG96_Hydrophobic',
 'ARG96_HBDonor',
 'ARG96_HBAcceptor',
 'ARG96_Anionic',
 'GLY123_Hydrophobic',
 'ASN124_Hydrophobic',
 'HIS125_Hydrophobic',
 'HIS125_PiStacking',
 'HIS125_EdgeToFace',
 'GLU126_Hydrophobic',
 'CYS127_Hydrophobic',
 'SER129_Hydrophobic',
 'ILE130_Hydrophobic',
 'ILE133_Hydrophobic',
 'TYR134_Hydrophobic',
 'TYR134_HBDonor',
 'TYR134_HBAcceptor',
 'VAL195_Hydrophobic',
 'PRO196_Hydrophobic',
 'ASP197_Hydrophobic',
 'CYS202_Hydrophobic',
 'CYS202_HBAcceptor',
 'TRP206_Hydrophobic',
 'TRP206_PiStacking',
 'TRP206_EdgeToFace',
 'ASP208_Hydrophobic',
 'ASP208_HBDonor',
 'ASP208_Cationic',
 'ASP220_Hydrophobic',
 'ASP220_HBDonor',
 'ASP220_Cationic',
 'ARG221_Hydrophobic',
 'ARG221_HBDonor',
 'ARG221_HBAcceptor',
 'ARG221_Anionic',
 'GLY222_Hydrophobic',
 'GLY222_HBDonor',
 'VAL223_Hydrophobic',
 'VAL223_HBDonor',
 'SER224_Hydrophobic',
 'SER224_HBDonor',
 'THR226_Hydrophobic',
 'THR226_HBDonor',
 'HIS248_

Generate new df with ordered columns

In [31]:
df_order = df_new[sorted_cols]

save ordered df to file for further processing

In [33]:
df_order.to_csv(outpath + "ligand_" + str(ligand) + "_res_based_in_columns_merged.csv")

Correct indices of computed df to only look at well equilibrated part of the simulation. Therefore, the first 30 ns of each simulation are cut off.

In [ ]:
df = pd.read_csv(outpath + "ligand_" + str(ligand) + "_res_based_in_columns_merged.csv")

df.drop(index=df.index[:3000], axis=0, inplace=True)
df = df.reset_index(drop=True)
df.drop(index=df.index[25000:28000], axis=0, inplace=True)
df = df.reset_index(drop=True)
df.drop(index=df.index[50000:53000], axis=0, inplace=True)
df = df.reset_index(drop=True)
print(len(df))

df.to_csv("ligand_"+str(ligand)+"_res_based_in_columns_merged_corrected.csv",sep=',')
